In [ ]:
# 아나콘다 가상환경 office:  py37TF2
# home : py37Keras

In [5]:
import nltk
import re
import numpy as np
import pandas as pd
import gensim
from nltk.tokenize import sent_tokenize
import multiprocessing
import io
from gensim.models import Phrases
from textblob import TextBlob
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import defaultdict

In [6]:
def character(text):

    essay_input_corpus = str(text) #문장입력
    essay_input_corpus = essay_input_corpus.lower()#소문자 변환

    sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
    total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
    total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
    
    split_sentences = []
    for sentence in sentences:
        processed = re.sub("[^a-zA-Z]"," ", sentence)
        words = processed.split()
        split_sentences.append(words)

    skip_gram = 1
    workers = multiprocessing.cpu_count()
    bigram_transformer = Phrases(split_sentences)

    model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

    model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
    
    #모델 설계 완료

    #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
    character_list = ['i', 'my', 'me', 'mine', 'you', 'your', 'they','them',
                      'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                    'brother','cousin','daughter','father','grandchild','granddaughter','granddson','grandfather',
                    'grandmother', 'person','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                    'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                    'twin','uncle','widow','widower','wife','ex-wife']
    
    ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
    #우선 토큰화한다.
    retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
    token_input_text = retokenize.tokenize(essay_input_corpus)
    #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
    #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
    filtered_chr_text = []
    for k in token_input_text:
        for j in character_list:
            if k == j:
                filtered_chr_text.append(j)
    
    #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
    
    filtered_chr_text_ = set(filtered_chr_text) #중복제거
    filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
    #print (filtered_chr_text__) # 중복값 제거 확인
    
    for i in filtered_chr_text__:
        ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
    
    char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
    char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
        
    result_char_ratio = round(char_total_count/total_words * 100, 2)

    return result_char_ratio, total_sentences, total_words, char_total_count, char_count_, ext_sim_words_key
    #return result_char_ratio


# 1. person-name-recognition

In [10]:
from keras.models import Model
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
#모델 불러오기
model = load_model('model-person-name-recognition.h5')

ImportError: cannot import name 'get_registered_name' from 'tensorflow.python.keras.utils.generic_utils' (/Users/kimkwangil/opt/anaconda3/envs/py37TF2/lib/python3.7/site-packages/tensorflow/python/keras/utils/generic_utils.py)

# Getting raw input names

In [11]:
with open('cleaned_inputs.txt', 'r') as f: # office mac에 데이터저장되어 있음
    raw_text = f.read()
    raw_inputs = raw_text.split('\n')

# Creating dictionary of characters

In [12]:
char2id = dict((c,i) for (i, c) in enumerate(sorted(set(raw_text.replace('\n', ' '))),1))
id2char = dict(enumerate(sorted(set(raw_text.replace('\n', ' '))),1))
char2id['<PAD>'] = 0
id2char[0] = '<PAD>'

In [13]:
label_dict = ['Non person name', 'Person Name']

In [14]:
# 이름 인식은 잘 됨, 단 모델을 학습시키고, 데이터베이스가 있어야 함, 이름의 첫 대문자도 구분함

test_string = 'Richard '

test_predict = np.expand_dims([char2id[char] for char in test_string],0)
test_predict = pad_sequences(test_predict, maxlen=40, padding='post')

output = model.predict(test_predict)
print(test_string,'is a',label_dict[np.argmax(output)])

NameError: name 'pad_sequences' is not defined

# 2.개체명인식으로 Named Entity Recognition 

In [15]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [16]:
sentence = "European authorities fined Google a record $5.1 billion on Wednesday for abusing its power \
            in the mobile phone market and ordered the company to alter its practices"
sentence_pos = pos_tag(word_tokenize(sentence))
print(sentence_pos) # 토큰화와 품사 태깅을 동시 수행

[('European', 'JJ'), ('authorities', 'NNS'), ('fined', 'VBD'), ('Google', 'NNP'), ('a', 'DT'), ('record', 'NN'), ('$', '$'), ('5.1', 'CD'), ('billion', 'CD'), ('on', 'IN'), ('Wednesday', 'NNP'), ('for', 'IN'), ('abusing', 'VBG'), ('its', 'PRP$'), ('power', 'NN'), ('in', 'IN'), ('the', 'DT'), ('mobile', 'JJ'), ('phone', 'NN'), ('market', 'NN'), ('and', 'CC'), ('ordered', 'VBD'), ('the', 'DT'), ('company', 'NN'), ('to', 'TO'), ('alter', 'VB'), ('its', 'PRP$'), ('practices', 'NNS')]


# 3.spaCy NER

참고 :  https://realpython.com/natural-language-processing-spacy-python/

In [58]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [59]:
input_text = """Bloomington Normal is almost laughably cliché for a midwestern city. Vast swathes of corn envelop winding roads and the heady smell of BBQ smoke pervades the countryside every summer. Yet, underlying the trite norms of Normal is the prescriptive force of tradition—the expectation to fulfill my role as a female Filipino by playing Debussy in the yearly piano festival and enrolling in multivariable calculus instead of political philosophy.So when I discovered the technical demand of bebop, the triplet groove, and the intricacies of chordal harmony after ten years of grueling classical piano, I was fascinated by the music's novelty. Jazz guitar was not only evocative and creative, but also strangely liberating. I began to explore different pedagogical methods, transcribe solos from the greats, and experiment with various approaches until my own unique sound began to develop. And, although I did not know what would be the 'best' route for me to follow as a musician, the freedom to forge whatever path I felt was right seemed to be exactly what I needed; there were no expectations for me to continue in any particular way—only the way that suited my own desires.While journeying this trail, I found myself at Interlochen Arts Camp the summer before my junior year. Never before had I been immersed in an environment so conducive to musical growth: I was surrounded by people intensely passionate about pursuing all kinds of art with no regard for ideas of what art 'should' be. I knew immediately that this would be a perfect opportunity to cultivate my sound, unbounded by the limits of confining tradition. On the first day of camp, I found that my peer guitarist in big band was another Filipino girl from Illinois. Until that moment, my endeavors in jazz guitar had been a solitary effort; I had no one with whom to collaborate and no one against whom I could compare myself, much less someone from a background mirroring my own. I was eager to play with her, but while I quickly recognized a slew of differences between us—different heights, guitars, and even playing styles—others seemed to have trouble making that distinction during performances. Some even went as far as calling me 'other-Francesca.' Thus, amidst the glittering lakes and musky pine needles of Interlochen, I once again confronted Bloomington's frustrating expectations.After being mistaken for her several times, I could not help but view Francesca as a standard of what the 'female Filipino jazz guitarist' should embody. Her improvisatory language, comping style and even personal qualities loomed above me as something I had to live up to. Nevertheless, as Francesca and I continued to play together, it was not long before we connected through our creative pursuit. In time, I learned to draw inspiration from her instead of feeling pressured to follow whatever precedent I thought she set. I found that I grew because of, rather than in spite of, her presence; I could find solace in our similarities and even a sense of comfort in an unfamiliar environment without being trapped by expectation. Though the pressure to conform was still present—and will likely remain present in my life no matter what genre I'm playing or what pursuits I engage in—I learned to eschew its corrosive influence and enjoy the rewards that it brings. While my encounter with Francesca at first sparked a feeling of pressure to conform in a setting where I never thought I would feel its presence, it also carried the warmth of finding someone with whom I could connect. Like the admittedly trite conditions of my hometown, the resemblances between us provided comfort to me through their familiarity. I ultimately found that I can embrace this warmth while still rejecting the pressure to succumb to expectations, and that, in the careful balance between these elements, I can grow in a way that feels both like discove"""

In [64]:
#소문자로 변환
input_lower_text = input_text.lower()
input_lower_text

"bloomington normal is almost laughably cliché for a midwestern city. vast swathes of corn envelop winding roads and the heady smell of bbq smoke pervades the countryside every summer. yet, underlying the trite norms of normal is the prescriptive force of tradition—the expectation to fulfill my role as a female filipino by playing debussy in the yearly piano festival and enrolling in multivariable calculus instead of political philosophy.so when i discovered the technical demand of bebop, the triplet groove, and the intricacies of chordal harmony after ten years of grueling classical piano, i was fascinated by the music's novelty. jazz guitar was not only evocative and creative, but also strangely liberating. i began to explore different pedagogical methods, transcribe solos from the greats, and experiment with various approaches until my own unique sound began to develop. and, although i did not know what would be the 'best' route for me to follow as a musician, the freedom to forge w

In [66]:
about_doc = nlp(input_lower_text)

In [67]:
token_list = {}
for token in about_doc:
    #print (token, token.idx)
    token_list.setdefault(token, token.idx)

In [80]:
#값 확인
token_list

{bloomington: 0,
 normal: 12,
 is: 19,
 almost: 22,
 laughably: 29,
 cliché: 39,
 for: 46,
 a: 50,
 midwestern: 52,
 city: 63,
 .: 67,
 vast: 69,
 swathes: 74,
 of: 82,
 corn: 85,
 envelop: 90,
 winding: 98,
 roads: 106,
 and: 112,
 the: 116,
 heady: 120,
 smell: 126,
 of: 132,
 bbq: 135,
 smoke: 139,
 pervades: 145,
 the: 154,
 countryside: 158,
 every: 170,
 summer: 176,
 .: 182,
 yet: 184,
 ,: 187,
 underlying: 189,
 the: 200,
 trite: 204,
 norms: 210,
 of: 216,
 normal: 219,
 is: 226,
 the: 229,
 prescriptive: 233,
 force: 246,
 of: 252,
 tradition: 255,
 —: 264,
 the: 265,
 expectation: 269,
 to: 281,
 fulfill: 284,
 my: 292,
 role: 295,
 as: 300,
 a: 303,
 female: 305,
 filipino: 312,
 by: 321,
 playing: 324,
 debussy: 332,
 in: 340,
 the: 343,
 yearly: 347,
 piano: 354,
 festival: 360,
 and: 369,
 enrolling: 373,
 in: 383,
 multivariable: 386,
 calculus: 400,
 instead: 409,
 of: 417,
 political: 420,
 philosophy.so: 430,
 when: 444,
 i: 449,
 discovered: 451,
 the: 462,
 technic

In [90]:
type(token_list.keys())

dict_keys

In [109]:
li_doc = list(token_list.keys())
type(li_doc[0])

spacy.tokens.token.Token

In [116]:
#캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
i_character_list = ['i', 'my', 'me', 'mine']
#하나씩 꺼내서 유사한 단어를 찾아내서 새로운 리스트에 담아서 출력,
ext_i_characters = []
for i_itm in i_character_list:
    for k_ in li_doc:
        if i_itm == str(k_):
            ext_i_characters.append(i_itm)
#I 관련 캐릭터 표현하는 단어들의 총 개수        
len(ext_i_characters)

49

In [119]:
#확인해보기
ext_i_characters

['i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'my',
 'my',
 'my',
 'my',
 'my',
 'my',
 'my',
 'my',
 'my',
 'my',
 'my',
 'me',
 'me',
 'me',
 'me',
 'me']

In [118]:
#캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
you_character_list = ['you', 'your', 
                  'yours', 'he','him','his' 'she','her']
#하나씩 꺼내서 유사한 단어를 찾아내서 새로운 리스트에 담아서 출력,
ext_you_characters = []
for i_itm in you_character_list:
    for k_ in li_doc:
        if i_itm == str(k_):
            ext_you_characters.append(i_itm)
#I 관련 캐릭터 표현하는 단어들의 총 개수        
len(ext_you_characters)

5

In [120]:
#확인해보기
ext_you_characters

['her', 'her', 'her', 'her', 'her']

In [55]:
token_list.values()

dict_values([0, 12, 19, 22, 29, 39, 46, 50, 52, 63, 67, 69, 74, 82, 85, 90, 98, 106, 112, 116, 120, 126, 132, 135, 139, 145, 154, 158, 170, 176, 182, 184, 187, 189, 200, 204, 210, 216, 219, 226, 229, 233, 246, 252, 255, 264, 265, 269, 281, 284, 292, 295, 300, 303, 305, 312, 321, 324, 332, 340, 343, 347, 354, 360, 369, 373, 383, 386, 400, 409, 417, 420, 430, 440, 441, 444, 449, 451, 462, 466, 476, 483, 486, 491, 493, 497, 505, 511, 513, 517, 521, 533, 536, 544, 552, 558, 562, 568, 571, 580, 590, 595, 597, 599, 603, 614, 617, 621, 626, 629, 636, 638, 643, 650, 654, 658, 663, 673, 677, 685, 687, 691, 696, 706, 716, 718, 720, 726, 729, 737, 747, 759, 766, 768, 779, 785, 790, 794, 800, 802, 806, 817, 822, 830, 841, 847, 850, 854, 861, 867, 873, 876, 883, 885, 888, 890, 899, 901, 905, 909, 914, 919, 925, 928, 932, 933, 937, 939, 945, 949, 952, 955, 962, 965, 967, 975, 977, 981, 989, 992, 998, 1007, 1012, 1014, 1019, 1023, 1029, 1036, 1039, 1042, 1050, 1055, 1057, 1063, 1065, 1071, 1076, 1079

In [31]:
def find_persons(text):
     # Create Doc object
     doc2 = nlp(text)

     # Identify the persons
     persons = [ent.text for ent in doc2.ents if ent.label_ == 'PERSON']

     # Return persons
     return persons

In [32]:
def find_I(text):
     # Create Doc object
     doc2 = nlp(text)

     # Identify the persons
     persons = [ent.text for ent in doc2.ents if ent.label_ == 'PERSON']

     # Return persons
     return persons

In [33]:
# 사람이 검출된다.
dtc_person = find_persons(input_text)
dtc_person

['Debussy', 'Francesca', 'Francesca', 'Francesca']

In [34]:
# 이제 패턴으로 특정 단어를 추출해보자
import spacy
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_lg")
matcher = PhraseMatcher(nlp.vocab)
terms = ["i", "you"]
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in terms]
matcher.add("i", patterns)

doc = nlp(input_text)
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)

In [35]:
doc = nlp(input_text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Bloomington Normal 0 18 ORG
every summer 170 182 DATE
Filipino 312 320 LANGUAGE
Debussy 332 339 PERSON
yearly 347 353 DATE
ten years 558 567 DATE
the first day 1623 1636 DATE
Filipino 1701 1709 NORP
Illinois 1720 1728 GPE
Interlochen 2281 2292 GPE
Bloomington 2318 2329 GPE
Francesca 2427 2436 PERSON
Filipino 2471 2479 NORP
Francesca 2648 2657 PERSON
Francesca 3348 3357 PERSON
first 3361 3366 ORDINAL


In [26]:
input_text = """Bloomington Normal is almost laughably cliché for a midwestern city. Vast swathes of corn envelop winding roads and the heady smell of BBQ smoke pervades the countryside every summer. Yet, underlying the trite norms of Normal is the prescriptive force of tradition—the expectation to fulfill my role as a female Filipino by playing Debussy in the yearly piano festival and enrolling in multivariable calculus instead of political philosophy.So when I discovered the technical demand of bebop, the triplet groove, and the intricacies of chordal harmony after ten years of grueling classical piano, I was fascinated by the music's novelty. Jazz guitar was not only evocative and creative, but also strangely liberating. I began to explore different pedagogical methods, transcribe solos from the greats, and experiment with various approaches until my own unique sound began to develop. And, although I did not know what would be the 'best' route for me to follow as a musician, the freedom to forge whatever path I felt was right seemed to be exactly what I needed; there were no expectations for me to continue in any particular way—only the way that suited my own desires.While journeying this trail, I found myself at Interlochen Arts Camp the summer before my junior year. Never before had I been immersed in an environment so conducive to musical growth: I was surrounded by people intensely passionate about pursuing all kinds of art with no regard for ideas of what art 'should' be. I knew immediately that this would be a perfect opportunity to cultivate my sound, unbounded by the limits of confining tradition. On the first day of camp, I found that my peer guitarist in big band was another Filipino girl from Illinois. Until that moment, my endeavors in jazz guitar had been a solitary effort; I had no one with whom to collaborate and no one against whom I could compare myself, much less someone from a background mirroring my own. I was eager to play with her, but while I quickly recognized a slew of differences between us—different heights, guitars, and even playing styles—others seemed to have trouble making that distinction during performances. Some even went as far as calling me 'other-Francesca.' Thus, amidst the glittering lakes and musky pine needles of Interlochen, I once again confronted Bloomington's frustrating expectations.After being mistaken for her several times, I could not help but view Francesca as a standard of what the 'female Filipino jazz guitarist' should embody. Her improvisatory language, comping style and even personal qualities loomed above me as something I had to live up to. Nevertheless, as Francesca and I continued to play together, it was not long before we connected through our creative pursuit. In time, I learned to draw inspiration from her instead of feeling pressured to follow whatever precedent I thought she set. I found that I grew because of, rather than in spite of, her presence; I could find solace in our similarities and even a sense of comfort in an unfamiliar environment without being trapped by expectation. Though the pressure to conform was still present—and will likely remain present in my life no matter what genre I'm playing or what pursuits I engage in—I learned to eschew its corrosive influence and enjoy the rewards that it brings. While my encounter with Francesca at first sparked a feeling of pressure to conform in a setting where I never thought I would feel its presence, it also carried the warmth of finding someone with whom I could connect. Like the admittedly trite conditions of my hometown, the resemblances between us provided comfort to me through their familiarity. I ultimately found that I can embrace this warmth while still rejecting the pressure to succumb to expectations, and that, in the careful balance between these elements, I can grow in a way that feels both like discove"""

In [27]:
character_ratio_result = character(input_text)
character_ratio_result
print("전체 문장에서 캐릭터를 의미하는 단어나 유사어 비율 :", character_ratio_result)

전체 문장에서 캐릭터를 의미하는 단어나 유사어 비율 : (8.79, 21, 705, 62, 8, [('also', 0.9799678325653076), ('feel', 0.9774074554443359), ('together', 0.9741417765617371), ('rewards', 0.9704742431640625), ('carried', 0.9699242115020752), ('play', 0.9693466424942017), ('continued', 0.9683886766433716), ('long', 0.9680884480476379), ('its', 0.9672611355781555), ('enjoy', 0.9666262269020081)])


/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:53: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
